Fetching authors and keywords

This is notebook that does fetching of author IDs and keywords of a paper based on its DOI using OpenAlex. This notebook is downloaded from https://github.com/SoftwareUnderstanding/SoftwareDisambiguationBenchmark/blob/56bf84109d4be642a16037e5cf8cdd9862e7aa7e/enrichment_notebooks/Use_CZI_mentions.ipynb.


In [4]:
import pandas as pd
import numpy as np
import requests
pd.set_option('max_colwidth', 1000)


In [5]:
# given a DOI, get the topics that are relevant from OpenAlex
# We get the topics over 0.5 to get the most relevants only
def getTopicsAndAuthors(doi):
    # Set the URL for the OpenAlex API
    url = "https://api.openalex.org/works/https://doi.org/"
    # Set the headers
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
    }
    # Set the parameters for the query
    #params = {
    #    'query': 'your_query_here',  # Replace 'your_query_here' with your actual query
    #    'apikey': 'your_api_key_here',  # Replace 'your_api_key_here' with your actual API key
    #}
    response = requests.get(url+str(doi), headers=headers)
    json_response = response.json()
    return_value = {"doi":doi}
    if(json_response["concepts"] is not None):
        return_list = []
        for concept in json_response["concepts"]:
            if(concept["score"]> 0.5):
                return_list.append(concept["display_name"])
    return_value["concepts"] = return_list
    if(json_response["authorships"] is not None):
        return_authors = []
        return_authors_openAlex = []
        for author in json_response["authorships"]:
            if(author["author"]):
                a = author["author"]
                return_authors_openAlex.append(a["id"])
                return_authors.append(a["display_name"])
    return_value["authors"] = return_authors
    return_value["authors_openAlex"] = return_authors_openAlex
    return return_value
    

In [6]:


# Load the CSV file
csv_path = "D:/MASTER/TMF/Software-Disambiguation/corpus/pypi_sampled.csv"  
df = pd.read_csv(csv_path,delimiter=";")
results = df['doi'].apply(getTopicsAndAuthors)

    # Extract data and add new columns
df['authors_oa'] = results.apply(lambda x: ','.join(x.get('authors_openAlex', [])) if isinstance(x, dict) else '')
df['authors'] = results.apply(lambda x: ','.join(x.get('authors', [])) if isinstance(x, dict) else '')
df['field/topic/keywords'] = results.apply(lambda x: ','.join(x.get('concepts', [])) if isinstance(x, dict) else '')

# Save the updated dataframe
updated_csv_path = "D:/MASTER/TMF/Software-Disambiguation/corpus/pypi_sampled.csv"
df.to_csv(updated_csv_path, index=False)


